In [1]:
import os
import sys

cwd = os.getcwd().split('/')
sys.path.append(os.path.join('/', *cwd[:cwd.index('discretizer')+1]))

In [2]:
from discretizer import momentum_operators
from discretizer.algorithms import magic

import sympy
import sympy.physics
from sympy.physics.quantum import TensorProduct as kr

In [3]:
sympy.init_printing(use_latex='mathjax')

In [4]:
s0 = sympy.eye(2)
sx = sympy.physics.matrices.msigma(1)
sy = sympy.physics.matrices.msigma(2)
sz = sympy.physics.matrices.msigma(3)

kx, ky, kz = momentum_operators
A, B, D, M, C = sympy.symbols('A B D M C', real=True)


In [5]:
k = sympy.sqrt(kx**2+ky**2)
hamiltonian = (+ C * kr(s0, s0) + M * kr(s0, sz) - D * k**2 * kr(s0, s0) 
               - B * k**2 * kr(s0, sz) + A * (kx * kr(sz, sx) - ky * kr(s0, sy)))

hamiltonian

⎡    ⎛  2      2⎞         ⎛  2      2⎞                                        
⎢- B⋅⎝kₓ  + k_y ⎠ + C - D⋅⎝kₓ  + k_y ⎠ + M              A⋅(kₓ + ⅈ⋅k_y)        
⎢                                                                             
⎢                                             ⎛  2      2⎞         ⎛  2      2
⎢             A⋅(kₓ - ⅈ⋅k_y)                B⋅⎝kₓ  + k_y ⎠ + C - D⋅⎝kₓ  + k_y 
⎢                                                                             
⎢                                                                             
⎢                    0                                         0              
⎢                                                                             
⎢                                                                             
⎣                    0                                         0              

                                                                              
                           0                       

In [11]:
symbolic = magic(hamiltonian, verbose=False, symbolic_output=True)

In [13]:
i, j = 0, 0
test = {key: val[i,j] for key, val in symbolic.items()}

In [14]:
test

⎧         B    D            B    D             4⋅B       4⋅D              B   
⎪(-1, 0): ── + ──, (0, -1): ── + ──, (0, 0): - ─── + C - ─── + M, (0, 1): ── +
⎨          2    2            2    2              2         2               2  
⎪         a    a            a    a              a         a               a   
⎩                                                                             

 D           B    D ⎫
 ──, (1, 0): ── + ──⎪
  2           2    2⎬
 a           a    a ⎪
                    ⎭